In [ ]:
import pickle
import numpy as np
import pyemma
import pyemma.plots as mplt
import matplotlib.pyplot as plt
import msmrd2.tools.trajectoryTools as trajectoryTools
import msmrd2.tools.analysis as analysisTools
import msmrd2.tools.msmTools as msmTools

In [ ]:
# Load parameters from parameters file
parentDirectory = '../data/dimer/60strength_improved/'
parameterDictionary = analysisTools.readParameters(parentDirectory + "parameters")
# Parameters for loading files (from original simulation)
nfiles = parameterDictionary['numFiles']
dt = parameterDictionary['dt'] 
stride = parameterDictionary['stride']
totalTimeSteps = parameterDictionary['timesteps'] 
parameterDictionary

In [ ]:
# Calculated parameters
dtEffective = dt*stride # needed to obtain rate dictionary
fnamebase = parentDirectory + 'simDimer_'
# Parameters for MSM generation 
numBoundStates = 8
lagtime = 400
reversible = True
stitching = True
fullDictionary = False # Only rates to and from bound states.

In [ ]:
# Load discrete trajectories
dtrajs = []
fnamesuffix = '_discrete_python' #'_discrete_python' # '_discrete'
filetype = 'xyz' #'xyz' # 'h5'
for i in range(nfiles):
    dtraj = trajectoryTools.loadDiscreteTrajectory(fnamebase, i, fnamesuffix, filetype)
    dtrajs.append(dtraj)    
    print("File ", i+1, " of ", nfiles, " done.", end="\r")

In [ ]:
# Slice trajectories getting rid of the unbound state 0
unboundStateIndex = 0
slicedDtrajs = trajectoryTools.splitDiscreteTrajs(dtrajs, unboundStateIndex)
# Stitch trajectories if wanted
if stitching:
    minLength = 5000
    finalTrajs = trajectoryTools.stitchTrajs(slicedDtrajs, minLength)
else:
    finalTrajs = slicedDtrajs

In [ ]:
# Create MSM between transision states and bound states without stitching
mainmsm = pyemma.msm.estimate_markov_model(finalTrajs, lagtime, reversible=reversible)
# The active set keep track of the indexes used by pyemma and the ones used to describe the state in our model.
activeSet = mainmsm.active_set
print(activeSet)

In [ ]:
# CK-test
nstates = 8
cktest = mainmsm.cktest(nstates, mlags=6)
mplt.plot_cktest(cktest);

In [ ]:
# Calculate implied time scales
maxlagtime = 500
nits = 20
its = pyemma.msm.its(finalTrajs, maxlagtime, reversible=reversible)

In [ ]:
# Plot implied timescales in ylog
mplt.plot_implied_timescales(its, nits = nits, ylog=True, units='steps', linewidth=2, dt=1)
plt.ylabel(r"log(timescale/steps)", fontsize = 24)
plt.xlabel(r"lag time/steps", fontsize = 24)
plt.ylim([10.0,8000])

In [ ]:
# Plot implied timescales without log
mplt.plot_implied_timescales(its, nits = nits, ylog=False, units='steps', linewidth=2, dt=1)
plt.ylabel(r"timescale/steps", fontsize = 24)
plt.xlabel(r"lag time/steps", fontsize = 24)

In [ ]:
eig0 = mainmsm.eigenvectors_right()[0]
print(np.where(eig0>0.1))
plt.plot(eig0)
plt.xlim([0,10])

In [ ]:
rateDictionary = msmTools.MSMtoRateDictionary(mainmsm, numBoundStates, dtEffective, fullDictionary)
# Alternatively, it can all be done with one function: slice and stitch trajectories then build MSM and extract rates
# rateDictionary = msmrdals.extractRatesMSM(dtrajs, lagtime, numBoundStates, stitching, fullDictionary)

In [ ]:
# Export to data file with pickle (to be loaded later)
pickle_out = open("../data/pickled_data/ratedictionary_dimer_t" + "{:.2E}".format(totalTimeSteps ) + "_s" + "{:d}".format(stride) + "_lagt" + "{:d}".format(lagtime) + ".pickle","wb")
pickle.dump(rateDictionary, pickle_out)
pickle_out.close()

In [ ]:
# Alternatively pickle MSM transition matrix and active set as a dictionary   
MSM = {'transition_matrix' : mainmsm.transition_matrix, 'active_set': mainmsm.active_set}
pickle_out = open("../data/pickled_data/MSM_dimer_t" + "{:.2E}".format(totalTimeSteps ) + "_s" + "{:d}".format(stride) + "_lagt" + "{:d}".format(lagtime) + ".pickle","wb")
pickle.dump(MSM, pickle_out)
pickle_out.close()

In [ ]:
MSM[1]

In [ ]:
rateDictionary

In [ ]:
rateDictionary["b2->b1"]